**Service Titan Internship**

**Applicant: Anna Shaljyan**

In [43]:
# Installing all the required packages if they are missing
# !pip install pandas
# !pip install pickle
# !pip install regex

In [44]:
# Importing all the required packages
import pandas as pd
import pickle
import re

In [45]:
# Loading invoices data from the pickle file
with open('invoices_new.pkl', 'rb') as f:
    invoices_data = pickle.load(f)

# Displaying the first invoice to inspect the structure
print(invoices_data[0])

{'id': '365371O', 'created_on': '2019-05-27', 'items': [{'item': {'id': 142446, 'name': 'ii_142446', 'unit_price': 182, 'type': 1}, 'quantity': 6}, {'item': {'id': 154019, 'name': 'ii_154019', 'unit_price': 176, 'type': 1}, 'quantity': 1}]}


In [54]:
# Loading expired IDs from the text file
with open('expired_invoices.txt', 'r') as f:
    expired_ids = []
    for line in f:
        # Split the line by commas and strip whitespace
        ids = line.strip().split(',')
        expired_ids.extend([int(id_str.strip()) for id_str in ids if id_str.strip().isdigit()])

# Displaying the expired IDs to inspect
print(expired_ids)

[305869, 377307, 391273, 385290, 331902, 397723, 343695, 340601, 347510, 325156, 379387, 348894, 322229, 326649, 377960, 325063, 369378, 319405, 383681, 367288, 356552, 394428, 381476, 381457, 349879, 343254, 366751, 330931, 363263, 326452, 379687, 356532, 383235, 313012, 368913, 352442, 315960, 351096, 331193, 392657, 352391, 389528, 338547, 379961, 337140, 323231, 371205, 378746, 307175, 340299]


In [55]:
class DataExtractor:
    def __init__(self, invoices_file, expired_ids_file):
        self.invoices_file = invoices_file
        self.expired_ids_file = expired_ids_file
        self.invoices_data = None
        self.expired_ids = None
        self.df = None

    def load_data(self):
      # Loading invoices data from pickle file
      with open(self.invoices_file, 'rb') as f:
        self.invoices_data = pickle.load(f)

    # Loading expired IDs from text file
      with open(self.expired_ids_file, 'r') as f:
        self.expired_ids = []
        for line in f:
            # Split the line by commas, strip whitespace, and convert to integer
            ids = line.strip().split(',')
            self.expired_ids.extend([int(id_str.strip()) for id_str in ids if id_str.strip().isdigit()])


    def transform_data(self):
        # Defining the type conversion table
        type_conversion = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other', 'O': "Material"}

        # Preparing a list to hold the flat data
        flat_data = []

        # Defining a dictionary to convert string numbers to integers
        number_words = {
            "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
            "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9, "ten": 10
        }

        def convert_quantity(quantity):
            if isinstance(quantity, str):
                return number_words.get(quantity.lower(), quantity)
            return quantity

        def correct_date(date_str):
            if date_str == '2019-02-30':
                return '2019-02-28'

            return date_str

        def extract_integer_from_string(s):
          # Replacing 'O' with '0' and then use regex to find all digit characters in the string
          s = str(s)
          s = s.replace('O', '0')
          digits = re.findall(r'\d+', s)
          # Joining all found digit groups (in case there are multiple groups of digits)
          digit_str = ''.join(digits)
          # Converting the resulting string of digits to an integer
          return int(digit_str)

        # Iterating over invoices
        for invoice in self.invoices_data:
            invoice_id = invoice['id']

            invoice_id = extract_integer_from_string(invoice_id)
            created_on = invoice['created_on']

            created_on = correct_date(created_on)

            # Checking if the 'items' key exists in the invoice
            if 'items' not in invoice:
                continue

            invoice_total = sum(item['item']['unit_price'] * convert_quantity(item['quantity']) for item in invoice['items'])
            is_expired = invoice_id in self.expired_ids

            # Iterating over invoice items
            for item in invoice['items']:
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                item_type = type_conversion[item['item']['type']]
                unit_price = item['item']['unit_price']
                quantity = item['quantity']
                total_price = unit_price * convert_quantity(quantity)
                percentage_in_invoice = total_price / invoice_total

                flat_data.append({
                    'invoice_id': invoice_id,
                    'created_on': pd.to_datetime(created_on),
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': item_type,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })

        # Creating a DataFrame from the flat data
        self.df = pd.DataFrame(flat_data)

    def process(self):
        self.load_data()
        self.transform_data()
        self.df.sort_values(by=['invoice_id', 'invoiceitem_id'], inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        return self.df

data_extractor = DataExtractor('invoices_new.pkl', 'expired_invoices.txt')
df = data_extractor.process()
df.head()

,invoice_id,created_on,invoiceitem_id,invoiceitem_name,type,unit_price,total_price,percentage_in_invoice,is_expired
0,301695,2019-04-26,103215,ii_103215,Other,135,945,0.374851,False
1,301695,2019-04-26,166227,ii_166227,Equipment,118,708,0.280841,False
2,301695,2019-04-26,171394,ii_171394,Other,128,128,0.050774,False
3,301695,2019-04-26,195625,ii_195625,Other,148,740,0.293534,False
4,304245,2019-03-17,121446,ii_121446,Service,158,632,0.128429,False


In [56]:
# Checking if the data has all the correct data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 9 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   invoice_id             492 non-null    int64         
 1   created_on             492 non-null    datetime64[ns]
 2   invoiceitem_id         492 non-null    int64         
 3   invoiceitem_name       492 non-null    object        
 4   type                   492 non-null    object        
 5   unit_price             492 non-null    int64         
 6   total_price            492 non-null    int64         
 7   percentage_in_invoice  492 non-null    float64       
 8   is_expired             492 non-null    bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(4), object(2)
memory usage: 31.4+ KB


In [57]:
df.to_csv('output.csv', index=False)